#CNN İLE TRAFİK KAZA PROJESİ

veri yükleme ve hazırlama

In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# CSV dosyasını okuma
df = pd.read_csv('dangerLevelLabel.csv')

# Veri artırma ve hazırlama
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

df['label'] = df['label'].astype(str)


# Eğitim ve doğrulama veri setlerini hazırlama
train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory='datapatch',
    x_col='filename',
    y_col='label',
    target_size=(28, 28),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    subset='training'
)

validation_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory='datapatch',
    x_col='filename',
    y_col='label',
    target_size=(28, 28),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    subset='validation'
)


model oluşturma

In [ ]:
# Model oluşturma fonksiyonu
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


def create_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 3)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # Binary classification
    ])
    return model

# Veri artırma fonksiyonu
def create_datagen():
    return ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

# Modeli derleme fonksiyonu
def compile_model(model):
    model.compile(
        loss='binary_crossentropy', 
        optimizer=Adam(learning_rate=0.0005),
        metrics=['accuracy']
    )

# Callback'lerin oluşturulması
def create_callbacks():
    early_stopping = EarlyStopping(monitor='val_loss', patience=5)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001)
    return [early_stopping, reduce_lr]

# Modeli ve veri artırma stratejilerini oluşturma
model = create_model()
datagen = create_datagen()
compile_model(model)
callbacks = create_callbacks()

# Veri artırma ile veri yükleme
train_datagen = datagen.flow_from_directory(
    'datapatch', 
    target_size=(28, 28),
    batch_size=32,
    class_mode='binary'  # kaza varsa 1, yoksa 0
)

validation_datagen = datagen.flow_from_directory(
    'datapatch',
    target_size=(28, 28),
    batch_size=32,
    class_mode='binary'  # kaza varsa 1, yoksa 0
)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



eğitme

In [ ]:
# Model eğitimi
history = model.fit(
    train_datagen,
    steps_per_epoch=train_datagen.n // 32,
    epochs=50,
    validation_data=validation_datagen,
    validation_steps=validation_datagen.n // 32,
    callbacks=callbacks
)
  